In [6]:
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings('ignore')

metadata = pd.read_csv('./the-movies-dataset/movies_metadata.csv', low_memory=False)
metadata.head(5)
#weighted rating of movies
C = metadata['vote_average'].mean()
print("weighted rating of movies: " + str(C))
# Calculate the minimum number of votes required to be in the chart, m
m = metadata['vote_count'].quantile(0.50)
print("minimum number of votes required to be in the chart: " + str(m))
#https://www.datacamp.com/community/tutorials/recommender-systems-python
# Filter out all qualified movies into a new DataFrame
q_movies = metadata.copy().loc[metadata['vote_count'] >= m]
q_movies.shape

# Function that computes the weighted rating of each movie
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula 
    return (v/(v+m) * R) + (m/(m+v) * C) # thing to modify for courses?
#adding new column with score
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)
#sorting movvies based on ^ and printing
q_movies = q_movies.sort_values('score', ascending=False)
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(15)
#this is a simple algorithm which recommends the top items based on percentile. In this case, line 12 

weighted rating of movies: 5.618207215133889
minimum number of votes required to be in the chart: 10.0


In [38]:
#Collecting data and formatting it. 
#metadata.head(5)
import urllib.request, urllib.parse, urllib.error
import json
import time
import random
import requests
import sys, time



## dd/mm/yyyy format
date = time.strftime("%m/%d/%Y")
courserequested = input(str("Enter the course code: "))
#getAllCourses = 'https://api.umd.io/v0/'+ courserequested+'/list'
#print(getAllCourses)

def getCourseURL(courseInput):
    url_return = 'https://api.umd.io/v0/courses/'
    url = 'https://api.umd.io/v0/courses/list'
    uh = urllib.request.urlopen(url)
    print('retrieving', url)
    data = uh.read().decode()
    info = json.loads(data)
    #print(json.dumps(info, indent=4, separators=(". ", " = ")))
    courses_array = []
    for courselist in info:
        if courselist['course_id'].startswith(courseInput):
            courses_array.append(courselist['course_id'])
            #print(courselist['course_id'])
            #print(courses_array)
    myString = ",".join(courses_array)
    #print(myString)  
    url_return=url_return+myString
    return url_return

#print(getCourseURL(courserequested))

def getJSON(url):
    uh = urllib.request.urlopen(url)
    print('retrieving', url)
    data = uh.read().decode()
    info = json.loads(data)
    return (json.dumps(info, indent=4, separators=(". ", " = ")))

print(getJSON(getCourseURL(courserequested)))


"""






#myString = ",".join(courses_array)
#print(myString)



for x in range(1):
    n = random.randint(1,4596)
    n = int(n)

course_id = info[n]['course_id']
name = info[n]['name']
cotd = 'The course of '+ date + ' is: '
tweets = cotd+course_id+ " " + name
t = tweets.replace("(","").replace(")","").replace("'","").replace(",","")



def main():
    print('tweeted')
while True:
    tweet()
time.sleep(1000)  # 1 sec

if __name__ == "__main__":
    main()
"""



retrieving https://api.umd.io/v0/courses/list
retrieving https://api.umd.io/v0/courses/CMSC122,CMSC131,CMSC132,CMSC132A,CMSC216,CMSC216H,CMSC250,CMSC250H,CMSC298A,CMSC320,CMSC330,CMSC351,CMSC389N,CMSC390,CMSC396H,CMSC411,CMSC412,CMSC414,CMSC417,CMSC420,CMSC421,CMSC422,CMSC423,CMSC424,CMSC425,CMSC426,CMSC433,CMSC434,CMSC435,CMSC436,CMSC451,CMSC452,CMSC456,CMSC457,CMSC460,CMSC466,CMSC470,CMSC474,CMSC475,CMSC498A,CMSC498L,CMSC498O,CMSC498U,CMSC499A,CMSC631,CMSC642,CMSC644,CMSC661,CMSC664,CMSC712,CMSC722,CMSC726,CMSC733,CMSC740,CMSC764,CMSC773,CMSC798,CMSC798F,CMSC799,CMSC818N,CMSC818W,CMSC818X,CMSC818Y,CMSC828B,CMSC828F,CMSC828U,CMSC828X,CMSC838J,CMSC858M,CMSC898,CMSC899
[
    {
        "course_id" = "CMSC122". 
        "name" = "Introduction to Computer Programming via the Web". 
        "dept_id" = "CMSC". 
        "department" = "Computer Science". 
        "semester" = "201901". 
        "credits" = "3". 
        "grading_method" = [
            "Regular"
        ]. 
        "core" = 

'\n\n\n\n\n\n\n#myString = ",".join(courses_array)\n#print(myString)\n\n\n\nfor x in range(1):\n    n = random.randint(1,4596)\n    n = int(n)\n\ncourse_id = info[n][\'course_id\']\nname = info[n][\'name\']\ncotd = \'The course of \'+ date + \' is: \'\ntweets = cotd+course_id+ " " + name\nt = tweets.replace("(","").replace(")","").replace("\'","").replace(",","")\n\n\n\ndef main():\n    print(\'tweeted\')\nwhile True:\n    tweet()\ntime.sleep(1000)  # 1 sec\n\nif __name__ == "__main__":\n    main()\n'

In [26]:
'''



retrieving https://api.umd.io/v0/courses/list
[
    {
        "course_id" = "AASP100". 
        "name" = "Introduction to African American Studies". 
        "department" = "African American Studies"
    }. 
    {
        "course_id" = "AASP100H". 
        "name" = "Introduction to African American Studies". 
        "department" = "African American Studies"
    }. 
    {
        "course_id" = "AASP101". 
        "name" = "Public Policy and the Black Community". 
        "department" = "African American Studies"
    }. 
]



'''

'\n\n\n\nretrieving https://api.umd.io/v0/courses/list\n[\n    {\n        "course_id" = "AASP100". \n        "name" = "Introduction to African American Studies". \n        "department" = "African American Studies"\n    }. \n    {\n        "course_id" = "AASP100H". \n        "name" = "Introduction to African American Studies". \n        "department" = "African American Studies"\n    }. \n    {\n        "course_id" = "AASP101". \n        "name" = "Public Policy and the Black Community". \n        "department" = "African American Studies"\n    }. \n]\n\n\n\n'